In [18]:
import pandas as pd

data_address = pd.read_csv("../data/지하철_주소_위경도_통합_all.csv", encoding = 'euc-kr')
data_address

,역명,주소,위도,경도
0,서울,서울특별시 중구 세종대로 지하2(남대문로 5가),37.553150,126.972533
1,시청,서울특별시 중구 세종대로 지하101(정동),37.563590,126.975407
2,종각,서울특별시 종로구 종로 지하55(종로1가),37.570203,126.983116
3,종로3가,서울특별시 종로구 종로 지하129(종로3가),37.570429,126.992095
4,종로5가,서울특별시 종로구 종로 지하216(종로5가),37.570971,127.001900
...,...,...,...,...
387,금릉,경기 파주시 금릉역로 85,37.751339,126.765410
388,금촌,경기 파주시 새꽃로 193,37.766162,126.774620
389,월롱,경기 파주시 월롱면 통일로 980,37.796215,126.792586
390,파주,경기 파주시 파주읍 주라위길 38,37.815366,126.792691


In [19]:
result = data_address[data_address['역명'].str.contains('\(.*\)', regex=True)]

# 결과 출력
print("[괄호가 포함된 역명 전체]")
print(result[['역명']])


[괄호가 포함된 역명 전체]
           역명
313  관악산(서울대)


<>:1: SyntaxWarning: invalid escape sequence '\('
<>:1: SyntaxWarning: invalid escape sequence '\('
/tmp/ipykernel_7682/2514821611.py:1: SyntaxWarning: invalid escape sequence '\('
  result = data_address[data_address['역명'].str.contains('\(.*\)', regex=True)]


In [20]:
import pandas as pd

data_address_latlng = pd.read_csv("../data/지하철_주소_위경도_통합_all.csv", encoding = 'euc-kr')
data_address_latlng

,역명,주소,위도,경도
0,서울,서울특별시 중구 세종대로 지하2(남대문로 5가),37.553150,126.972533
1,시청,서울특별시 중구 세종대로 지하101(정동),37.563590,126.975407
2,종각,서울특별시 종로구 종로 지하55(종로1가),37.570203,126.983116
3,종로3가,서울특별시 종로구 종로 지하129(종로3가),37.570429,126.992095
4,종로5가,서울특별시 종로구 종로 지하216(종로5가),37.570971,127.001900
...,...,...,...,...
387,금릉,경기 파주시 금릉역로 85,37.751339,126.765410
388,금촌,경기 파주시 새꽃로 193,37.766162,126.774620
389,월롱,경기 파주시 월롱면 통일로 980,37.796215,126.792586
390,파주,경기 파주시 파주읍 주라위길 38,37.815366,126.792691


In [42]:
import folium
import pandas as pd

# CSV 파일에서 지하철 역 데이터 불러오기
data_address_latlng = pd.read_csv("../data/지하철_1_8호선 역주소(위경도) 좌표 정보.csv", encoding = 'euc-kr') # 파일명은 알맞게 수정

# 지도 중심 좌표 설정 (서울 시청 기준)
m = folium.Map(location=[37.5665, 126.9780], zoom_start=12)

# 각 지하철역에 대해 Circle 마커 생성
for _, row in data_address_latlng.iterrows():
    lat = row['위도']
    lon = row['경도']
    station_name = str(row['역명'])[:2]  # 역 이름 앞 2글자
    folium.Circle(
        location=[lat, lon],
        radius=500,  # 반지름 (단위: meter)
        color='skyblue',
        fill=True,
        fill_color='skyblue',
        fill_opacity=0.1,
        popup=station_name
    ).add_to(m)
m

# HTML로 저장 또는 노트북에서 직접 출력

m.save("../data/지하철역_지도.html")


In [43]:
import pandas as pd
import folium

# 1. 파일 불러오기
coord_df = pd.read_csv("../data/지하철_주소_위경도_통합_all.csv", encoding='euc-kr')

# 2. 필요 컬럼만 추출
coord_df = coord_df[['역명', '주소', '위도', '경도']]

# ✅ 3. NaN 값 제거 (주소, 위도, 경도 중 하나라도 없으면 제외)
coord_df = coord_df.dropna(subset=['주소', '위도', '경도'])

# ✅ 4. '주소'가 '서울특별시'로 시작하는 행만 필터링
coord_df = coord_df[coord_df['주소'].str.startswith('서울특별시', na=False)]

# 5. folium 지도 생성
m = folium.Map(location=[37.5665, 126.9780], zoom_start=12)

# 6. Circle 마커 추가
for _, row in coord_df.iterrows():
    lat = row['위도']
    lon = row['경도']
    station_name = str(row['역명'])[:2]  # 앞 두 글자만 표시
    folium.Circle(
        location=[lat, lon],
        radius=500,
        color='skyblue',
        fill=True,
        fill_color='skyblue',
        fill_opacity=0.2,
        popup=station_name
    ).add_to(m)

# 7. 지도 출력 또는 저장
m.save("../data/지하철역_지도_OnlySeoul.html")



In [40]:
import pandas as pd
import folium

# 1. 파일 불러오기
toprank_df = pd.read_csv("../data/subway_complexity_toprank_대표호선.csv", encoding='utf-8')
# 1. 파일 불러오기
coord_df = pd.read_csv("../data/지하철_주소_위경도_통합_all.csv", encoding='euc-kr')

# 2. 필요 컬럼만 추출
coord_df = coord_df[['역명', '주소', '위도', '경도']]

# ✅ 3. NaN 값 제거 (주소, 위도, 경도 중 하나라도 없으면 제외)
coord_df = coord_df.dropna(subset=['주소', '위도', '경도'])

# ✅ 4. '주소'가 '서울특별시'로 시작하는 행만 필터링
coord_df = coord_df[coord_df['주소'].str.startswith('서울특별시', na=False)]

# 5. 출근 + 퇴근 합계 컬럼 추가
toprank_df['total_commute'] = toprank_df['morning'] + toprank_df['evening']

# 6. total_commute 기준 상위 30개 역 추출
top30_df = toprank_df.sort_values(by='total_commute', ascending=False).head(30)

# 7. 좌표와 병합 (역명 기준)
final_df = pd.merge(top30_df, coord_df, on='역명', how='inner')

# 8. 지도 생성
m = folium.Map(location=[37.5665, 126.9780], zoom_start=12)

# 9. 마커 표시
for _, row in final_df.iterrows():
    lat = row['위도']
    lon = row['경도']
    station_name = row['역명']
    morning = row['morning']
    evening = row['evening']
    total_commute = row['total_commute']
    
    # 색상 조건
    if total_commute >= 1200000:
        color = 'red'
    elif total_commute >= 1000000:
        color = 'orange'
    else:
        color = 'green'

    popup_text = f"""<b>{station_name}</b><br>
    출근: {morning:,}명<br>
    퇴근: {evening:,}명<br>
    합계: {total_commute:,}명"""

    folium.Circle(
        location=[lat, lon],
        radius=600,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.1,
        popup=folium.Popup(popup_text, max_width=300)
    ).add_to(m)

# 10. 저장 (선택)
m.save("../data/서울_출퇴근_혼잡도_top30.html")

